In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import seaborn as sns
import cufflinks as cf
import plotly.express as px
from scipy import stats
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline() 


import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
df.head()

In [ ]:
df_quality =df.groupby(['quality'])['quality'].count().reset_index(name='quality Count').sort_values(by= 'quality Count', ascending= False)
df_quality

In [ ]:
px.pie(df_quality, values='quality Count', names= 'quality', title= 'Quality of Wines' )

In [ ]:
df.columns

Attention: we have imbalnced classification data and should first ttry to balance data before starting with forecasting methods

# Distribution of Features

In [ ]:
fig, ax = plt.subplots(6,2, figsize=(12,8))
axes_ = [axes_row for axes in ax for axes_row in axes]

for i, j in enumerate(df.columns):
    g = sns.boxplot(x = df[j], ax = axes_[i])
    g.set_title(j)
    plt.tight_layout()

In [ ]:
#Heatmap: correlation of features and Quality Classifiactions¶
fig = plt.figure(figsize=(20,10))
sns.heatmap(df.corr(), annot = True, fmt = '.2f')

we can conclude from heatmap that following Features have almost zero correlation with Quality of the wine and can be removed from further analysis :

'pH'

'residual sugar'

'free sulfur dioxide'

In [ ]:
df = df.drop(columns=['pH','residual sugar','free sulfur dioxide'])

In [ ]:
df.columns

# Features Analysis

1. outliers check
2. BoxCox Check (check if the variance of the feature is almost constant and the data can be normal distributed)
3. Scatter disrtibution of features vs quality

In [ ]:
# Outliers check
def check_outliers(data, col):
    
    #calculating quantiles
    q25 = data[col].quantile(q = 0.25)
    q75 = data[col].quantile(q = 0.75)
    
    #calculating inter quantile range
    iqr = q75-q25
    
    #calculating upper_bound and lower_bound
    lower_bound = q25 - (1.5*iqr)
    upper_bound = q75 + (1.5*iqr)
    
    #filtering out outliers....!!!!!
    outliers = data[col][(data[col] < lower_bound) | (data[col] > upper_bound)]
    
    
    print("**** Printing Outliers Result ****")
    
    print("\nTotal Outliers Present in the Data: %s"%(len(outliers)))
    
    #plotting the line plot result
    #plt.figure(figsize=(10,10))
    plt.plot(data[col])
    plt.title(col + " with Outliers")
    plt.scatter(x = outliers.index, y = outliers.values, marker = "X", color = 'r', s = 100)
    
    

In [ ]:
data=df['fixed acidity']
col = df.columns
BoXCox_Data,fitted_lambda = stats.boxcox(data)

figure=plt.figure(figsize= (15,5))
plt.subplot(221)
sns.distplot(data , color='green')
plt.subplot(222)
sns.distplot(BoXCox_Data , color='blue' )
plt.subplot(223)
sns.stripplot(x= 'quality', y= data, data= df, jitter= True, hue='quality') 
plt.legend().remove()
plt.subplot(224)
check_outliers(df, col[1])
figure. tight_layout(pad=3.0)


In [ ]:
data=df['volatile acidity']
col = df.columns
BoXCox_Data,fitted_lambda = stats.boxcox(data)

figure=plt.figure(figsize= (15,5))
plt.subplot(221)
sns.distplot(data , color='green')
plt.subplot(222)
sns.distplot(BoXCox_Data , color='blue' )
plt.subplot(223)
sns.stripplot(x= 'quality', y= data, data= df, jitter= True, hue='quality') 
plt.legend().remove()
plt.subplot(224)
check_outliers(df, col[2])
figure. tight_layout(pad=3.0)

In [ ]:
data=df['citric acid']
col = df.columns
BoXCox_Data,fitted_lambda = stats.boxcox(data-min(data)+1)

figure=plt.figure(figsize= (15,5))
plt.subplot(221)
sns.distplot(data , color='green')
plt.subplot(222)
sns.distplot(BoXCox_Data , color='blue' )
plt.subplot(223)
sns.stripplot(x= 'quality', y= data, data= df, jitter= True, hue='quality') 
plt.legend().remove()
plt.subplot(224)
check_outliers(df, col[3])
figure. tight_layout(pad=3.0)

Too Many Outliers!!

In [ ]:
data=df['chlorides']
col = df.columns
BoXCox_Data,fitted_lambda = stats.boxcox(data-min(data)+1)

figure=plt.figure(figsize= (15,5))
plt.subplot(221)
sns.distplot(data , color='green')
plt.subplot(222)
sns.distplot(BoXCox_Data , color='blue' )
plt.subplot(223)
sns.stripplot(x= 'quality', y= data, data= df, jitter= True, hue='quality') 
plt.legend().remove()
plt.subplot(224)
check_outliers(df, col[4])
figure. tight_layout(pad=3.0)

In [ ]:

data=df['total sulfur dioxide']
col = df.columns
BoXCox_Data,fitted_lambda = stats.boxcox(data-min(data)+1)

figure=plt.figure(figsize= (15,5))
plt.subplot(221)
sns.distplot(data , color='green')
plt.subplot(222)
sns.distplot(BoXCox_Data , color='blue' )
plt.subplot(223)
sns.stripplot(x= 'quality', y= data, data= df, jitter= True, hue='quality') 
plt.legend().remove()
plt.subplot(224)
check_outliers(df, col[5])
figure. tight_layout(pad=3.0)

In [ ]:
    
data=df['density']
col = df.columns
BoXCox_Data,fitted_lambda = stats.boxcox(data-min(data)+1)

figure =plt.figure(figsize= (15,5))
plt.subplot(221)
sns.distplot(data , color='green')
plt.subplot(222)
sns.distplot(BoXCox_Data , color='blue' )
plt.subplot(223)
sns.stripplot(x= 'quality', y= data, data= df, jitter= True, hue='quality') 
plt.legend().remove()
plt.subplot(224)
check_outliers(df, col[6])

figure. tight_layout(pad=3.0)

In [ ]:
data=df['sulphates']
col = df.columns
BoXCox_Data,fitted_lambda = stats.boxcox(data-min(data)+1)

figure =plt.figure(figsize= (15,5))
plt.subplot(221)
sns.distplot(data , color='green')
plt.subplot(222)
sns.distplot(BoXCox_Data , color='blue' )
plt.subplot(223)
sns.stripplot(x= 'quality', y= data, data= df, jitter= True, hue='quality') 
plt.legend().remove()
plt.subplot(224)
check_outliers(df, col[7])

figure. tight_layout(pad=3.0)

In [ ]:
   
data=df['alcohol']
col = df.columns
BoXCox_Data,fitted_lambda = stats.boxcox(data-min(data)+1)

figure=plt.figure(figsize= (15,5))
plt.subplot(221)
sns.distplot(data , color='green')
plt.subplot(222)
sns.distplot(BoXCox_Data , color='blue' )
plt.subplot(223)
sns.stripplot(x= 'quality', y= data, data= df, jitter= True, hue='quality') 
plt.legend().remove()
plt.subplot(224)
check_outliers(df, col[8])
figure. tight_layout(pad=3.0)

# Forecasting

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler , PowerTransformer, QuantileTransformer, Normalizer , MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import *

In [ ]:
dict1 ={3 : 1, 4: 2, 5: 3, 6:4 , 7:5, 8: 6}

df['wine_Class'] =df['quality'].map(dict1)
df.head()

In [ ]:
df['wine_Class'].value_counts()

Imbalanced Data

Dealing with Imbalanced Data usinf SMOTE

In [ ]:
from sklearn.model_selection import train_test_split
X= df.drop(columns=['quality', 'wine_Class'])
y= df['wine_Class']

from imblearn.over_sampling import SMOTE
oversample = SMOTE(k_neighbors=6)
# transform the dataset
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
lgr_pipline_s  = Pipeline([("scaler", StandardScaler()), ("LogisticRegression", LogisticRegression())])
rfc_pipline_s = Pipeline([("scaler", StandardScaler()), ("RandomForestClassifier", RandomForestClassifier())])
knn_pipline_s = Pipeline([("scaler", StandardScaler()), ("KNeighborsClassifier", KNeighborsClassifier())])
gnb_pipline_s = Pipeline([("scaler", StandardScaler()), ("GaussianNB", GaussianNB())])
sgd_pipline_s = Pipeline([("scaler", StandardScaler()), ("SGDClassifier", SGDClassifier())])
dt_pipline_s = Pipeline([("scaler", StandardScaler()), ("DecisionTreeClassifier", DecisionTreeClassifier())])

In [ ]:
pip_dicts ={0:'Logistic Regression' , 1:'RandomForestClassifier' , 2: 'KNeighborsClassifier' ,
            3: 'GaussianNB', 4: 'SGDClassifier', 5: 'DecisionTreeClassifier' }

In [ ]:
pips=[lgr_pipline_s, rfc_pipline_s , knn_pipline_s, gnb_pipline_s  , sgd_pipline_s , dt_pipline_s ]

In [ ]:
scores_df = pd.DataFrame(columns = ["Model", "CVScores"])
for i, pipe in enumerate(pips):
    score = cross_val_score(pipe, X, y, cv = 10)
    print(pip_dicts[i], ": ", score.mean())

In [ ]:
grid_params = [
    {"classifier": [RandomForestClassifier()],
    "classifier__n_estimators": [50,100,150,200,250,300],
    "classifier__criterion": ["gini", "entropy"],
    "classifier__max_features": ["auto", "sqrt", "log2"],
    },
          
    
]

In [ ]:
pipeline_new = Pipeline([("scaler", StandardScaler()), ("classifier", RandomForestClassifier())])

random_search = RandomizedSearchCV(estimator = pipeline_new, param_distributions = grid_params, scoring = 'neg_mean_absolute_error', n_jobs= -1, cv = 8, verbose = 10, random_state = 42)


In [ ]:
best_model = random_search.fit(X_train, y_train)

In [ ]:
best_model.best_params_

In [ ]:
rfc_pipline_s  = Pipeline([("scaler", StandardScaler()), ("RandomForestClassifier", RandomForestClassifier(max_features='sqrt', n_estimators=150))])

In [ ]:
model = rfc_pipline_s.fit(X_train, y_train)

In [ ]:
predict = model.predict(X_test)



In [ ]:
from sklearn import metrics
print('Accuracy Score for train dataset : ' , metrics.accuracy_score(y_test, predict))
print('Accuracy Score for test dataset : ' , metrics.accuracy_score(y_test, predict))

In [ ]:
conn_cm_test = metrics.confusion_matrix(y_test, predict, [1,0])
sns.heatmap(conn_cm_test, fmt= '.2f', annot=True)